# **Taller 1 de Sistemas Basados en Geolocalización**

## Introducción
Este taller tiene como objetivo aplicar conceptos de geolocalización y cálculo de distancias utilizando la fórmula de Haversine. Además, se explorará la variación del área de la Laguna de Fúquene a lo largo del tiempo para evaluar su impacto en la delimitación de terrenos y desplazamiento de poblaciones. Los resultados deberán ser sustentados mediante mapas dinámicos y análisis comparativos.

---

## Profesor: Carlos Armando López Solano, MSc.

### 1. Cálculo de distancias con la fórmula de Haversine
Desarrolle un sistema de software que, mediante el uso de la fórmula de Haversine, calcule la distancia en cada caso:

| Caso | Φ₁ (Latitud 1) | λ₁ (Longitud 1) | Φ₂ (Latitud 2) | λ₂ (Longitud 2) |
|------|---------------|---------------|---------------|---------------|
| 1    | 4.6369566546922485 | -74.08344303563474 | 4.698387290122269 | -74.08913229761984 |
| 2    | 2.4930822620498403 | -76.56469927139997 | 4.553288632232513 | -75.65871130171998 |
| 3    | 7.141088134111742 | -73.11869057856512 | 42.35927858183055 | -71.09328053009632 |

Luego, compare el resultado obtenido con la medición de dicha distancia utilizando el mapa dinámico de su preferencia.

#### Preguntas:
- ¿Qué ubicaciones corresponden a las coordenadas dadas?
- ¿Qué conclusión puede sacar de estas mediciones?

---

In [17]:
# Import Required Libraries

# Import folium library for creating maps
import folium
import math

In [ ]:
class Coordenada:
    def __init__(self,city:str,name:str,latitude:float,longitude:float):
        self.city = city
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        
    def get_latitude(self):
        return self.latitude
    
    def get_longitude(self):
        return self.longitude
    
    def __str__(self):
        return f"City: {self.city}, Name: {self.name}, Latitude: {self.latitude}, Longitude: {self.longitude}"
    

class TierraPlana:
    def __init__(self):
        self.coordenadas = []
        
    def agregar_coordenada(self, coordenada:Coordenada):
        self.coordenadas.append(coordenada)
        
    def distancia_entre_coordenadas(self,coordenada1:Coordenada,coordenada2:Coordenada) -> float:
        return self.haversine(coordenada1.get_latitude(),coordenada1.get_longitude(),coordenada2.get_latitude(),coordenada2.get_longitude())
    
    def get_coordenada(self,name:str) -> Coordenada:     
        for coordenada in self.coordenadas:
            if coordenada.name == name:
                return coordenada
        return None

    def radio_tierra_en_latitud(self, lat:float) -> float:
        a = 6378.137  #Radius at sea level at equator
        b = 6356.752  #Radius at poles
        
        c = (a**2*math.cos(lat))**2
        d = (b**2*math.sin(lat))**2
        e = (a*math.cos(lat))**2
        f = (b*math.sin(lat))**2
        
        return math.sqrt((c+d)/(e+f))

    def haversine(self, Lat1:float,Lon1:float,lat2:float,lon2:float) -> float:
        # El radio lo calculamos con el prodmedio de las latitudes
        R = self.radio_tierra_en_latitud((Lat1+lat2)/2)
        
        # Calculamos la formula de haversine
        dLat = math.radians(lat2 - Lat1)
        dLon = math.radians(lon2 - Lon1)
        a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(math.radians(Lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2) * math.sin(dLon/2)
        c = 2*R*math.atan(math.sqrt(a))
        return c
    
    def mostrar_coordenadas(self):
        for coordenada in self.coordenadas:
            print(coordenada)

class Mapa:
    def crear_mapa(self,coordenada1:Coordenada,coordenada2:Coordenada):
        
        distancia_coordenadas = TierraPlana().distancia_entre_coordenadas(coordenada1,coordenada2)
        if distancia_coordenadas < 100:
            zoom = 12
        elif distancia_coordenadas < 300:
            zoom = 8
        else:
            zoom = 4
        
        map_osm = folium.Map(location=[coordenada1.get_latitude(), coordenada1.get_longitude()], zoom_start=zoom)
        
        folium.Marker(
            location=[coordenada1.get_latitude(), coordenada1.get_longitude()],
            popup=f"{coordenada1.city} - {coordenada1.name}"
        ).add_to(map_osm)

        folium.Marker(
            location=[coordenada2.get_latitude(), coordenada2.get_longitude()],
            popup=f"{coordenada2.city} - {coordenada2.name}"
        ).add_to(map_osm)
        
        folium.PolyLine(
            locations=[[coordenada1.get_latitude(),coordenada1.get_longitude()],[coordenada2.get_latitude(),coordenada2.get_longitude()]],
            color="#FF0000",
            weight=5,
            tooltip="Distancia: {:.2f} km".format(),
        ).add_to(map_osm)
        return map_osm



In [83]:
casos = [
    {"city": "Bogota", "name":"Universidad Nacional", "latitude": 4.6369566546922485, "longitude": -74.08344303563474 },
    {"city": "Bogota", "name":"Universidad Uniminuto", "latitude": 4.698387290122269, "longitude": -74.08913229761984 },
    {"city": "Popayan", "name":"Casa de mi EX", "latitude": 2.4930822620498403, "longitude": -76.56469927139997},
    {"city": "Armenia", "name":"Universidad del Quindio", "latitude": 4.553288632232513, "longitude": -75.65871130171998},
    {"city": "Bucaramanga", "name":"Universidad industrial de Santander", "latitude": 4.553288632232513, "longitude": -75.65871130171998},
    {"city": "Boston", "name":"Massachusetts institute of technology", "latitude": 42.35927858183055 , "longitude": -71.09328053009632}

]

tierra_plana = TierraPlana()

coordenadas = []

for caso in casos:
    coordenada = Coordenada(caso["city"],caso["name"], caso["latitude"], caso["longitude"])
    tierra_plana.agregar_coordenada(coordenada)


### **Caso 1**

In [92]:
cordenada1 = tierra_plana.get_coordenada("Universidad Nacional")
cordenada2 = tierra_plana.get_coordenada("Universidad Uniminuto")
distancia_coordenadas = tierra_plana.distancia_entre_coordenadas(cordenada1,cordenada2) 

print(f"La distancia entre las siguientes coordenadas es {distancia_coordenadas:.2f} KM")
print(cordenada1)
print(cordenada2)

miMapa = Mapa()
miMapa.crear_mapa(cordenada1,cordenada2)

La distancia entre las siguientes coordenadas es 6.84 KM
City: Bogota, Name: Universidad Nacional, Latitude: 4.6369566546922485, Longitude: -74.08344303563474
City: Bogota, Name: Universidad Uniminuto, Latitude: 4.698387290122269, Longitude: -74.08913229761984


### **Caso 2**

In [95]:
cordenada3 = tierra_plana.get_coordenada("Casa de mi EX")
cordenada4 = tierra_plana.get_coordenada("Universidad del Quindio")
distancia_coordenadas = tierra_plana.distancia_entre_coordenadas(cordenada3,cordenada4) 

print(f"La distancia entre las siguientes coordenadas es {distancia_coordenadas:.2f} KM")
print(cordenada3)
print(cordenada4)

miMapa = Mapa()
miMapa.crear_mapa(cordenada3,cordenada4)

La distancia entre las siguientes coordenadas es 250.29 KM
City: Popayan, Name: Casa de mi EX, Latitude: 2.4930822620498403, Longitude: -76.56469927139997
City: Armenia, Name: Universidad del Quindio, Latitude: 4.553288632232513, Longitude: -75.65871130171998


### **Caso 3**

In [90]:
cordenada5 = tierra_plana.get_coordenada("Universidad industrial de Santander")
cordenada6 = tierra_plana.get_coordenada("Massachusetts institute of technology")
distancia_coordenadas = tierra_plana.distancia_entre_coordenadas(cordenada5,cordenada6) 

print(f"La distancia entre las siguientes coordenadas es {distancia_coordenadas:.2f} KM")
print(cordenada5)
print(cordenada6)

miMapa = Mapa()
miMapa.crear_mapa(cordenada5,cordenada6)

La distancia entre las siguientes coordenadas es 4003.94 KM
City: Bucaramanga, Name: Universidad industrial de Santander, Latitude: 4.553288632232513, Longitude: -75.65871130171998
City: Boston, Name: Massachusetts institute of technology, Latitude: 42.35927858183055, Longitude: -71.09328053009632


In [ ]:
# Set Up the Map

# Create a map object centered at a specific location using folium
# Latitude and Longitude for the center of the map Bogota
latitude = 4.60971
longitude = -74.08175 
# Create a folium map object
map_osm = folium.Map(location=[latitude, longitude], zoom_start=12)

# Display the map
map_osm


### 2. Análisis del área de la Laguna de Fúquene
La Laguna de Fúquene tiene un área que varía cada dos décadas, contrayéndose y expandiéndose.

Para ayudar a los gobernadores de Boyacá y Cundinamarca a tomar decisiones sobre el desplazamiento de campesinos y la delimitación de fincas, se solicita determinar el porcentaje del área de crecimiento de la laguna (diferencia entre las áreas máxima y mínima). Pueden utilizar el método que deseen. 

#### Requisitos:
- Presentar el resultado final en un mapa dinámico.
- Sustentar el análisis con datos y metodología utilizada.

---

## Bibliografía sugerida
- J. A. E. García Álvarez, “ASÍ FUNCIONA EL GPS,” 2015. [Online]. Available: [https://goo.gl/eXsp7K](https://goo.gl/eXsp7K). [Accessed: 25-Feb-2017].
- C. Beatty, “Location-Based Services: Navigation for the Masses, At Last!,” Journal of Navigation, 30-May-2002. [Online]. Available: [https://goo.gl/uXwEwI](https://goo.gl/uXwEwI). [Accessed: 12-Apr-2017].
